In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loader.TrainValLoader import train_val_loader
from loader.CSVLoader import csv_loader
from loader.TestLoader import test_loader
layers = 45
#load the data
gt,wave = csv_loader()
all_data = train_val_loader()
all_data_test = test_loader()

In [ ]:
import numpy as np
from prep.ResizeBlock import make_resize_block
from prep.HelperBlocks import pipe
from prep.SmoothDerivativeBlock import savgol_block

#define processing for images
transform_set1 = lambda d, is_test: pipe(d,[
    make_resize_block(16, pre_fill=0, order=0),
    lambda d: savgol_block(d,5,2,1,2),
    lambda d: np.ma.filled(d,0)
]) 

transform_set2 = lambda d, is_test: pipe(d,[
    make_resize_block(24, pre_fill=0, order=0),
    lambda d: savgol_block(d,5,2,1,2),
    lambda d: np.ma.filled(d,0)
]) 

In [ ]:
#use layer selection methods in a following manner
from selMethods.SpearmanCor import spearman_method


transformed1, transform_next, indices = spearman_method(transform_set1(all_data,False),gt["K"],final_count=30)
transformed_test1 = transform_next(transform_set1(all_data_test,True))

transformed2, transform_next, indices = spearman_method(transform_set2(all_data,False),gt["P"],final_count=30)
transformed_test2 = transform_next(transform_set2(all_data_test,True))

In [ ]:
#save results
np.save(f"./experiment/test_K",transformed1)
np.save(f"./experiment/test_K_test",transformed_test1)